In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd


# Feature selection

This is the current outlook for the training data for our group:

Features:
- store_id
- plaace_hierarchy_id
- chain (true/false)
- mall (true/false)
- grunnkrets_id (merge)
- area_km2
- municpiality_name(?)
- num_buss_stops
- knute_punkt
- average_age
- couples_children_average (?)
- single_parent_average (?)
- all_households_income
- numBusStops
- revenue

### Starting by preprocessing each column to be added:

Changing chain_name and mall_name to true/false, whether they have a chain or is located in a mall or not.

In [2]:
stores_train = pd.read_csv('data/stores_train.csv')


stores_train = stores_train.rename(columns={"chain_name": "chain_bool", "mall_name": "mall_bool"})


stores_train.loc[stores_train["chain_bool"].isnull(), "chain_bool"] = 0
stores_train.loc[stores_train["mall_bool"].isnull(), "mall_bool"] = 0

stores_train.loc[stores_train["chain_bool"] != 0, "chain_bool"] = 1
stores_train.loc[stores_train["mall_bool"] != 0, "mall_bool"] = 1

print("Chain numbers:\n", stores_train["chain_bool"].value_counts())
print("Mall numbers:\n", stores_train["mall_bool"].value_counts())
stores_train.head()

Chain numbers:
 0    9122
1    3737
Name: chain_bool, dtype: int64
Mall numbers:
 0    10579
1     2280
Name: mall_bool, dtype: int64


,store_id,year,store_name,plaace_hierarchy_id,sales_channel_name,grunnkrets_id,address,lat,lon,chain_bool,mall_bool,revenue
0,983540538-974187930-44774,2016,MCDONALD'S BRAGERNES TORG MAGASINET,1.1.1.0,Hamburger restaurants,6020303,BRAGERNES TORG 13,59.743104,10.204928,1,1,17.998
1,987074191-973117734-44755,2016,MCDONALD'S KLINGENBERGGATA,1.1.1.0,Hamburger restaurants,3010306,NaN,59.913759,10.734031,1,0,23.828
2,984890265-981157303-64491,2016,BURGER KING HØNEFOSS,1.1.1.0,Hamburger restaurants,6050102,KONG RINGS GATE 1,60.164751,10.254656,1,1,16.099
3,914057442-992924179-126912,2016,BURGER KING GLASSHUSPASSASJEN,1.1.1.0,Hamburger restaurants,18040102,STORGATA 12,67.283669,14.379796,1,1,9.296
4,913018583-913063538-668469,2016,BURGER KING TILLERTORGET,1.1.1.0,Hamburger restaurants,16017414,NaN,63.358068,10.374832,1,1,4.528


### Merging avg age

In [55]:


grunnkrets_age = pd.read_csv("data/grunnkrets_age_distribution.csv")



print(grunnkrets_age.shape)
grunnkrets_age = grunnkrets_age[grunnkrets_age["year"]>2015] #only focus on the latest data available
grunnkrets_age = grunnkrets_age.drop(["year"],axis=1)








ages = np.array([int(c.split("_")[-1]) for c in grunnkrets_age.filter(regex="^age_\d+$").columns])
grunnkrets_age.loc[:, "avg_age"] = (grunnkrets_age.filter(regex="^age_\d+$").values * ages).sum(axis=1) / grunnkrets_age.filter(regex="^age_\d+$").values.sum(axis=1)





filtered_frame = grunnkrets_age[["grunnkrets_id","avg_age"]]

print(filtered_frame.sort_values(by="avg_age",ascending=False))

print(stores_train.shape)

stores_train = pd.merge(stores_train, filtered_frame, how="inner", on="grunnkrets_id")
print(stores_train.shape)


















(22620, 93)
       grunnkrets_id    avg_age
1134         2191704  81.190476
1129         1360305  79.349057
1137         2310601  74.953782
19099       11020114  74.410811
1135         2191803  73.341176
...              ...        ...
18552       19022038  23.379562
8634        16630505  23.095890
7411         2191202  22.677019
2199        18040720  22.642857
19961        2190818  21.694444

[11304 rows x 2 columns]
(24057, 17)
(24057, 18)


C:\Users\danny\AppData\Local\Temp\ipykernel_26764\1625815638.py:29: FutureWarning: Passing 'suffixes' which cause duplicate columns {'avg_age_x'} in the result is deprecated and will raise a MergeError in a future version.
  stores_train = pd.merge(stores_train, filtered_frame, how="inner", on="grunnkrets_id")


#### Merging stores_train with grunnkrets:

In [3]:
grunnkrets = pd.read_csv('data/grunnkrets_norway_stripped.csv')

print(grunnkrets.columns)

grunnkrets = grunnkrets.drop(["year"], axis=1)

stores_train = stores_train.drop(["year"], axis=1)
stores_train = pd.merge(stores_train, grunnkrets[["grunnkrets_id", "municipality_name", "geometry_gk", "area_km2"]], how="inner", on="grunnkrets_id")


stores_train = stores_train.drop_duplicates()

stores_train.head()

Index(['grunnkrets_id', 'year', 'grunnkrets_name', 'district_name',
       'municipality_name', 'geometry_gk', 'area_km2'],
      dtype='object')


,store_id,store_name,plaace_hierarchy_id,sales_channel_name,grunnkrets_id,address,lat,lon,chain_bool,mall_bool,revenue,municipality_name,geometry_gk,area_km2
0,983540538-974187930-44774,MCDONALD'S BRAGERNES TORG MAGASINET,1.1.1.0,Hamburger restaurants,6020303,BRAGERNES TORG 13,59.743104,10.204928,1,1,17.998,Drammen,"POLYGON((10.2046156903846 59.7447808519649, 10...",0.155779
2,916139195-998469740-540508,WASABI SUSHI,1.1.2.0,Sushi and wok restaurants,6020303,ENGENE 4,59.744230,10.206481,0,0,3.814,Drammen,"POLYGON((10.2046156903846 59.7447808519649, 10...",0.155779
4,999512895-999558380-491766,YAYAS DRAMMEN,1.1.3.0,Thai restaurants,6020303,ENGENE 6,59.744065,10.206848,1,0,4.016,Drammen,"POLYGON((10.2046156903846 59.7447808519649, 10...",0.155779
6,999539130-999586686-619467,TAJ MAHAL DRAMMEN AS,1.1.3.0,Thai restaurants,6020303,NEDRE STORGATE 23,59.742875,10.210364,0,0,4.482,Drammen,"POLYGON((10.2046156903846 59.7447808519649, 10...",0.155779
8,976377966-976442466-53817,JONAS B GUNDERSEN DRAMMEN,1.1.5.0,Pizza restaurants,6020303,BRAGERNES TORG 8,59.744258,10.205450,1,0,18.184,Drammen,"POLYGON((10.2046156903846 59.7447808519649, 10...",0.155779


#### Working with busstops

In [5]:
grunnkrets_geo = pd.read_csv("data/grunnkrets_norway_stripped.csv")
grunnkrets_geo = grunnkrets_geo.drop(["year"], axis=1)
grunnkrets_geo['geometry_gk'] = gpd.GeoSeries.from_wkt(grunnkrets_geo['geometry_gk'])
grunnkrets_geo = gpd.GeoDataFrame(grunnkrets_geo, geometry='geometry_gk')

busstops_geo = pd.read_csv('data/busstops_norway.csv')
busstops_geo['geometry_bus'] = gpd.GeoSeries.from_wkt(busstops_geo['geometry_bus'])
busstops_geo = gpd.GeoDataFrame(busstops_geo, geometry='geometry_bus')

dfsjoin = gpd.sjoin(grunnkrets_geo, busstops_geo)


AttributeError: module 'geopandas' has no attribute 'GeoSeries'

In [5]:
# dfpivot = pd.pivot_table(dfsjoin, index="grunnkrets_id", columns="busstop_id", aggfunc={"busstop_id":len})
# dfpivot.columns = dfpivot.columns.droplevel()

# print(dfsjoin["grunnkrets_id"].value_counts())

# test_df = dfsjoin.groupby(["grunnkrets_id"], as_index=False).count()

# print(test_df)

dfsjoin["busstop_count"] = dfsjoin.groupby(["grunnkrets_id"])['busstop_id'].transform('count')
dfsjoin = dfsjoin.drop_duplicates()

dfsjoin

,grunnkrets_id,grunnkrets_name,district_name,municipality_name,geometry_gk,area_km2,index_right,busstop_id,stopplace_type,importance_level,side_placement,busstop_count
0,10010701,Grim-Dueknipen,Grim-Møllevann-Dalane,Kristiansand,"MULTIPOLYGON (((7.97200 58.14337, 7.97210 58.1...",0.768961,66100,5159cea4-7668-4fd8-8dc8-19c898c22552,Kun skilt,Mangler viktighetsnivå,RIGHT,18
0,10010701,Grim-Dueknipen,Grim-Møllevann-Dalane,Kristiansand,"MULTIPOLYGON (((7.97200 58.14337, 7.97210 58.1...",0.768961,32001,93fa2615-5c7c-412a-8c0c-183cac028083,Plattform og lomme,Mangler viktighetsnivå,LEFT,18
0,10010701,Grim-Dueknipen,Grim-Møllevann-Dalane,Kristiansand,"MULTIPOLYGON (((7.97200 58.14337, 7.97210 58.1...",0.768961,65744,d6af8929-0847-4d58-9244-5090d8955724,Plattform og lomme,Mangler viktighetsnivå,RIGHT,18
0,10010701,Grim-Dueknipen,Grim-Møllevann-Dalane,Kristiansand,"MULTIPOLYGON (((7.97200 58.14337, 7.97210 58.1...",0.768961,32002,df2850e2-b26c-4ee7-8301-b31b00ceb721,Kantstopp,Mangler viktighetsnivå,LEFT,18
0,10010701,Grim-Dueknipen,Grim-Møllevann-Dalane,Kristiansand,"MULTIPOLYGON (((7.97200 58.14337, 7.97210 58.1...",0.768961,32003,67cb917f-5b02-4574-9611-f13e9ba028bd,Plattform og lomme,Mangler viktighetsnivå,LEFT,18
...,...,...,...,...,...,...,...,...,...,...,...,...
26060,11220301,Gilja,Indre Del,Gjesdal,"POLYGON ((6.25254 58.80573, 6.25256 58.80571, ...",2.769097,16330,b56a41d9-f7ca-49c3-b005-3d9463a5e9d3,NaN,Mangler viktighetsnivå,LEFT,12
26060,11220301,Gilja,Indre Del,Gjesdal,"POLYGON ((6.25254 58.80573, 6.25256 58.80571, ...",2.769097,49787,8924c193-8365-4c26-839d-aa61d1fc9132,Plattform og lomme,Lokalt knutepunkt,RIGHT,12
26060,11220301,Gilja,Indre Del,Gjesdal,"POLYGON ((6.25254 58.80573, 6.25256 58.80571, ...",2.769097,16329,0b304a06-0acc-4b0f-8c8b-6b888c23bacf,Plattform og lomme,Lokalt knutepunkt,LEFT,12
26060,11220301,Gilja,Indre Del,Gjesdal,"POLYGON ((6.25254 58.80573, 6.25256 58.80571, ...",2.769097,16354,a1e41ccf-2625-4069-b79c-a7c59eefd05e,NaN,Mangler viktighetsnivå,LEFT,12


In [7]:
pd.set_option('display.max_rows', 15)

knutepunkter = ("Lokalt knutepunkt", "Nasjonalt knutepunkt", "Regionalt knutepunkt")

# print(dfsjoin["importance_level"])

bus_data = dfsjoin.index.isin(dfsjoin[dfsjoin["importance_level"] == "Lokalt knutepunkt"].reset_index().groupby("grunnkrets_id").index.first())

print(bus_data)

dfsjoin.loc[bus_data, "knutepunkt_bool"] = 1

dfsjoin


[False False False ...  True  True  True]


,grunnkrets_id,grunnkrets_name,district_name,municipality_name,geometry_gk,area_km2,index_right,busstop_id,stopplace_type,importance_level,side_placement,busstop_count,knutepunkt_bool
0,10010701,Grim-Dueknipen,Grim-Møllevann-Dalane,Kristiansand,"MULTIPOLYGON (((7.97200 58.14337, 7.97210 58.1...",0.768961,66100,5159cea4-7668-4fd8-8dc8-19c898c22552,Kun skilt,Mangler viktighetsnivå,RIGHT,18,NaN
0,10010701,Grim-Dueknipen,Grim-Møllevann-Dalane,Kristiansand,"MULTIPOLYGON (((7.97200 58.14337, 7.97210 58.1...",0.768961,32001,93fa2615-5c7c-412a-8c0c-183cac028083,Plattform og lomme,Mangler viktighetsnivå,LEFT,18,NaN
0,10010701,Grim-Dueknipen,Grim-Møllevann-Dalane,Kristiansand,"MULTIPOLYGON (((7.97200 58.14337, 7.97210 58.1...",0.768961,65744,d6af8929-0847-4d58-9244-5090d8955724,Plattform og lomme,Mangler viktighetsnivå,RIGHT,18,NaN
0,10010701,Grim-Dueknipen,Grim-Møllevann-Dalane,Kristiansand,"MULTIPOLYGON (((7.97200 58.14337, 7.97210 58.1...",0.768961,32002,df2850e2-b26c-4ee7-8301-b31b00ceb721,Kantstopp,Mangler viktighetsnivå,LEFT,18,NaN
0,10010701,Grim-Dueknipen,Grim-Møllevann-Dalane,Kristiansand,"MULTIPOLYGON (((7.97200 58.14337, 7.97210 58.1...",0.768961,32003,67cb917f-5b02-4574-9611-f13e9ba028bd,Plattform og lomme,Mangler viktighetsnivå,LEFT,18,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
26060,11220301,Gilja,Indre Del,Gjesdal,"POLYGON ((6.25254 58.80573, 6.25256 58.80571, ...",2.769097,16330,b56a41d9-f7ca-49c3-b005-3d9463a5e9d3,NaN,Mangler viktighetsnivå,LEFT,12,1.0
26060,11220301,Gilja,Indre Del,Gjesdal,"POLYGON ((6.25254 58.80573, 6.25256 58.80571, ...",2.769097,49787,8924c193-8365-4c26-839d-aa61d1fc9132,Plattform og lomme,Lokalt knutepunkt,RIGHT,12,1.0
26060,11220301,Gilja,Indre Del,Gjesdal,"POLYGON ((6.25254 58.80573, 6.25256 58.80571, ...",2.769097,16329,0b304a06-0acc-4b0f-8c8b-6b888c23bacf,Plattform og lomme,Lokalt knutepunkt,LEFT,12,1.0
26060,11220301,Gilja,Indre Del,Gjesdal,"POLYGON ((6.25254 58.80573, 6.25256 58.80571, ...",2.769097,16354,a1e41ccf-2625-4069-b79c-a7c59eefd05e,NaN,Mangler viktighetsnivå,LEFT,12,1.0
